<a href="https://colab.research.google.com/github/petikmaryna/DSL/blob/main/job2PetikNEW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Створення прикладу грамматики:

In [ ]:
import copy
grammar = { 'toks': list(
                    [(0, ' '),
                     (11, 'aa'),
                     (12, 'bb'),
                     (21, 'ab'),
                     (22, 'cc')]
                    ),
            'vars': {
                'A': [['B', (11, 'aa')],
                      ['D'],
                      ['F', (12, 'bb')]],
                'B': [['B'],
                      ['F', (21, 'ab')]],
                'C': [['C'],
                      ['A',(22, 'cc')],
                      ['F', (11, 'aa')]],
                'D': [[(0, ' ')]],
                'F': [
                      ['B', (21, 'ab')],
                      ['F', (12, 'bb')]]
                'R': [[(12, 'bb')]]
                    },
            'hvar': 'A',
            }


**Необхідно написати такі функції:**

1) Функція для прибирання сторонніх нетерміналів.
Для того, щоб прибрати з грамматики усі сторонні символи, треба створити декілька алгоритмів.
Перший алгоритм, який прибирає непродуктивні нетермінали: у циклі перебираються нетермінали та відповідні їм правила. Якщо нетермінал не є непродуктивним, то він залишається і додається до нового словника(при цьому перевіряється, чи не існує вже в словарі такий), якщо є непродуктивним - ігнорується. Також перебираються правила, які відповідають продуктивним нетерміналам: якщо правило містить непродуктивний нетермінал, то воно відкидається.     


In [ ]:
def deleteExtSymb(grammar):
    NonExt = searchNonExtSymb(grammar)
    newGrammar = copy.deepcopy(grammar)
    oldVars = grammar['vars']
    toks = grammar['toks']
    newVars = dict()
    for varr, defin in oldVars.items():
         if varr in NonExt:
             for rules in defin:
                 if checkTokenZ(rules, toks, NonExt):
                    if varr in newVars.keys():
                        newVars[varr].append(rules)
                    else:
                        newVars[varr] = list()
                        newVars[varr].append(rules)
    newGrammar['vars'] = newVars
    return newGrammar
    

Продуктивні нетермінали визначаються допоміжною функцією, яка перевіряє, чи знаходяться в правилі у нетермінала тільки токени і/або вже відомі продуктивні нетермінали.

In [ ]:
def searchNonExtSymb(grammar):
    tokens = grammar['toks']
    variables = grammar['vars']
    NonExtSymb = list()
    flag = True
    while flag:
        flag = False
        for var, definition in variables.items():
            for rule in definition:
                if checkTokenZ(rule, tokens,NonExtSymb):
                    if var not in NonExtSymb:
                        NonExtSymb.append(var)
                        flag = True
    return NonExtSymb

Допоміжна функція, яка перевіряє правила.

In [ ]:
def checkTokenZ(rule, Tokens, someList):
    for partOfRule in rule:
        if partOfRule in Tokens or partOfRule in someList:
            pass
        else:
            return False
    return True

Вивід функції: 

```
{'toks': 
  [(0, ' '), 
  (11, 'aa'), 
  (12, 'bb'), 
  (21, 'ab'), 
  (22, 'cc')], 
'vars': 
{'A': [['D']], 
 'C': [['C'], ['A', (22, 'cc')]], 
 'D': [[(0, ' ')]],
 'R': [[(12, 'bb')]]
 }, 
 'hvar': 'A'}
```
Далі нам потрібно створити ще один алгоритм, який прибирає недосяжні нетермінали. Недосяжні нетермінали -- це такі, до яких не можна дістатися, починаючи зі стартового нетерміналу.
У циклі функції перебираються нетермінали, і якщо вони є досяжними, то додаються до списку нетерміналів для нової грамматики разом з правилами.

In [ ]:

def deleteNonReachable(grammar):
    reachblSymb = searchReachable(grammar)
    newVars = dict()
    tokens = grammar['toks']
    variables = grammar['vars']
    for var, definition in variables.items():
        if var in reachblSymb:
            for rule in definition:
                if checkToken(rule, var, reachblSymb):
                    if var in newVars.keys():
                        newVars[var].append(rule)
                else:
                    newVars[var] = list()
                    newVars[var].append(rule)
    print(newVars)
    grammar['vars'] = newVars
    return grammar
    

Допоміжна функція, яка шукає досяжні нетермінали. Спочатку досяжним є лише один нетермінал - стартовий. Потім до списку досяжних нетерміналів додається нетермінал, який знаходиться в правилах стартового нетерміналу, бо якщо нетермінал в лівій частині є досяжним, то і нетермінал в правій частині буде досяжним. Далі продивляються по черзі усі інші нетермінали.

In [ ]:

def searchReachable(grammar):
    reachableSymb = list() 
    tokens = grammar['toks']
    variables = grammar['vars']
    reachableSymb.append(grammar['hvar'])
    flag = True
    while flag:
        flag = False
        for var, definition in variables.items():
          if var in reachableSymb:
            for rule in definition:
              for elem in rule:
                if elem not in tokens and elem not in reachableSymb:
                  flag = True
                  reachableSymb.append(elem)
    return reachableSymb
    

In [ ]:
Вивід: 
{'toks': [(0, ' '), (11, 'aa'), (12, 'bb'), (21, 'ab'), (22, 'cc')], 'vars': {'A': [['D']], 'D': [[(0, ' ')]]}, 'hvar': 'A'}

Із граматики прибрані недосяжні і непродуктивні символи, тобто тепер вона позбавлена сторонніх символів.

2) Функція для визначення зникаючих символів.
В основу цієї функції полягли допоміжні функції з попереднього завдання(перетерпівши мінімальні зміни).
Візьмемо за зникаючий токен (0, ' '). Тоді функція для пошуку нетерміналів, які є зникаючими, має такий алгоритм: перебираються нетермінали і відокремлюються ті, які мають в своїх правилах або зникаючий токен, або/і зникаючий відомий нетермінал.


In [ ]:
def searchDisappSymb(grammar):
    tokens = grammar['toks']
    variables = grammar['vars']
    emptySymb = list()
    flag = True
    
    while flag:
        flag = False
        for var, definition in variables.items():
            for rule in definition:
                if checkToken(rule, tokens[0],emptySymb):
                    if var not in emptySymb:
                        emptySymb.append(var)
                        flag = True
    return emptySymb

Допоміжна функція виглядає так:

In [ ]:
def checkToken(rule, Token, someList):
    for partOfRule in rule:
        if partOfRule == Token or partOfRule in someList:
            pass
        else:
            return False
    return True

Вивід функції: 


```
['D', 'A']
```

